In [1]:
import pandas as pd
import numpy as np
from typing import Tuple, List, Dict
import matplotlib.pyplot as plt
import math
import random

from scipy.optimize import minimize

import sys
sys.path.insert(0, '../scripts/')
import utils as utl

In [2]:
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import math

class Decision_Tree_Classifier:
    def __init__(self) -> None:
        self.res_dict = {}
    
    def fit(self, X, y, X_cat_cols):
        self.res_dict = self.des_tree_nodes(X, y, X_cat_cols)
        
    def predict(self, X):
        return X.apply(lambda x: self.predict_single(x, self.res_dict), axis=1)

    def predict_single(self, x, res_dict):
        tp = res_dict['type']
        attr = res_dict['attr']
        if tp == 'cat':
            if x[attr] in res_dict['res'].keys():
                res = res_dict['res'][x[attr]]
            else:
                rn = random.randint(0, len(list(res_dict['res'].keys())) - 1)
                res = res_dict['res'][list(res_dict['res'].keys())[rn]]
        else:
            val = res_dict['vals']
            i = self.recover_index(x[attr],val)
            if str(i) in res_dict['res'].keys():
                res = res_dict['res'][str(i)]
            else:
                rn = random.randint(0, len(list(res_dict['res'].keys())) - 1)
                res = res_dict['res'][list(res_dict['res'].keys())[rn]]
            
        
        if isinstance(res,dict):
            return self.predict_single(x, res)
        else:
            return res

    def new_nodes(self, X: pd.DataFrame, y: pd.Series, attr_max_gain: str, X_cat_cols) -> Dict:
        dct_res = {}
        cols = list(X.columns)
        cols.remove(attr_max_gain)
        X_upd = X[cols]
        Xcc = X_cat_cols.copy()
        if attr_max_gain in X_cat_cols:
            Xcc.remove(attr_max_gain)
        
        dct_res['attr'] = attr_max_gain
        if attr_max_gain in X_cat_cols:
            dct_res['type'] = 'cat'
            dct_res['vals'] = np.sort(X[attr_max_gain].unique())
            childs = {}

            for u in dct_res['vals']:
                Xn = X_upd.loc[X[attr_max_gain]==u]
                yn = y.loc[X[attr_max_gain]==u]
                if len(yn)>0:
                    childs[u] = yn.iloc[0]
                    if len(yn.unique()) > 1:
                        childs[u] = self.des_tree_nodes(Xn, yn, Xcc)
            dct_res['res'] = childs
        else:
            dct_res['type'] = 'cont'
            splits = self.get_splits(X[attr_max_gain], y)
            dct_res['vals'] = splits
            childs = {}
            for i in range(len(splits) + 1):                
                Xn = self.iloc_ranges(X_upd, X[attr_max_gain], splits, i)
                yn = self.iloc_ranges(y, X[attr_max_gain], splits, i)
                if len(yn)>0:
                    childs[str(i)] = self.des_tree_nodes(Xn, yn, Xcc)
            dct_res['res'] = childs
        return dct_res
    
    def des_tree_nodes(self, X: pd.DataFrame, y: pd.Series, X_cat_cols):
        cols = list(X.columns)
        attr_max_gain = self.get_max_ig(X, y, X_cat_cols)
        cols.remove(attr_max_gain)
        if len(cols)!=0:
            out = self.new_nodes(X, y, attr_max_gain, X_cat_cols)
            return out
        y_count = {}
        for u in y.unique():
            y_count[u] = y.loc[y==u].count()
        out = max(y_count, key=y_count.get)
        return out
    
    def info_gain_cat(self, x:pd.Series, y:pd.Series) -> float:
        """Calculates the info gain of using a given categorical attribute to describe categorical data

        Parameters
        ----------
        x : pd.Series
            Categorical values of the attribute
        y : pd.Series
            Categorical target

        Returns
        -------
        float
            Information gain
        """
        ES = self.entropy(y)
        ESv = 0
        nt = len(y)
        if nt != 0:
            for u in np.sort(x.unique()):
                Sv = y.loc[x==u]
                n = len(Sv)        
                ESv += - (n/nt)*self.entropy(Sv)
        return ES - ESv

    def info_gain_con(self, x:pd.Series, y:pd.Series) -> float:
        """Calculates the info gain of using a given continuous attribute to describe categorical data

        Parameters
        ----------
        x : pd.Series
            Categorical values of the attribute
        y : pd.Series
            Categorical target

        Returns
        -------
        float
            Information gain
        """
        splits = self.get_splits(x, y)
        ES = self.entropy(y)
        ESv = 0
        nt = len(y)
        if nt != 0:
            for i in range(len(splits) + 1):
                Sv = self.iloc_ranges(y, x, splits, i)
                n = len(Sv)
                ESv += - (n/nt)*self.entropy(Sv)
        return ES - ESv

    def get_max_ig(self, X: pd.DataFrame, y: pd.Series, X_cat_cols: List) -> str:
        """Identifies the column of X with the maximum info gain related to the target y

        Parameters
        ----------
        X : pd.DataFrame
            Input data
        y : pd.Series
            Target data
        X_cat_cols : List
            List of categorical columns of X

        Returns
        -------
        str
            Name of the column with maximum information gain
        """
        dct = {}
        cols = list(X.columns)
        for attr in cols:
            if attr in X_cat_cols:
                dct[attr] = self.info_gain_cat(X[attr], y)
            else:
                dct[attr] = self.info_gain_con(X[attr], y)
        return max(dct, key=dct.get)
    
    def iloc_ranges(self, y: pd.Series, x: pd.Series, splits: List, i: int) -> pd.Series:
        """Filters the data from the series y for which the continuous data x is between the values split[i-1] and split[i]

        Parameters
        ----------
        x : pd.Series
            Continuous data to use as base for the filtering
        y : pd.Series or pd.DataFrame
            Data to be filtered
        splits : List
            Values at which the values of x can be splitted
        i : int
            Index of the split to be used


        Returns
        -------
        pd.Series
            Filtered data
        """
        if len(x) > 0:
            if i > 0 and i < len(splits):
                Sv = y.loc[(x>splits[i-1]) & (x<=splits[i])]
            elif i == 0:
                Sv = y.loc[x<=splits[0]]
            else:
                Sv = y.loc[x>splits[-1]]
            return Sv
        return pd.Series([])

    def recover_index(self, val: float, splits: List) -> int:
        """Recovers the index at which val falls whitin the range of the values specified by splits

        Parameters
        ----------
        val : pd.Series
            Value to evaluate
        splits : List
            Limits of the split ranges.

        Returns
        -------
        int
            Recovered index
        """
        for i,v in enumerate(splits):
            if i > 0:
                if val > splits[i-1] and val <= v:
                    return round(i)
            else:
                if val <= v:
                    return round(i)
        return round(len(splits))
    
    def get_splits(self, x:pd.Series, y:pd.Series) -> np.array:
        """Returns the splits of the categorical data x, using the median x for each category of y

        Parameters
        ----------
        x : pd.Series
            Input data for which the splits will be calculated
        y : pd.Series
            Target categorical data 

        Returns
        -------
        np.array
            Limits of the splits
        """
        vals = []
        for u in y.unique():
            x_np = x.loc[y==u].to_numpy()
            vals.append(np.median(x_np))
        vals_sorted = np.sort(np.unique(np.array(vals)))
        return vals_sorted
    
    def entropy(self, y: pd.Series) -> float:
        """Calculates the entropy of y

        Parameters
        ----------
        y : pd.Series
            Categorical value for which the entropy will be calculated

        Returns
        -------
        float
            Entropy
        """
        entropy = 0
        ntt = len(y)
        if ntt != 0:
            for u in y.unique():
                y_c = y.loc[y==u]
                p = len(y_c)/ntt
                if p != 0:
                    entropy += -p*math.log2(p)
        return entropy
    

class Decision_Tree_Regressor(Decision_Tree_Classifier):
    def __init__(self, min_n_data = None) -> None:
        super().__init__()
        if min_n_data is None:
            self.min_n_data = 50
        else:
            self.min_n_data = min_n_data

    def fit(self, 
            X, y):
        df_split = X.copy()
        df_split['res'] = y.copy()
        df_split['pred'] = y.copy()
        df_split = df_split.reset_index(drop = True)

        self.res_dict = self.des_tree_nodes(df_split)

    def des_tree_nodes(self, df: pd.DataFrame) -> Dict:
        dct_res = {}    
        attr_max_gain, x_v = self.get_min_mse(df)

        dct_res['type'] = 'cont'
        dct_res['attr'] = attr_max_gain
        splits = [x_v]
        dct_res['vals'] = splits

        childs = {}
        df_s = df.sort_values(attr_max_gain)
        df_low = df_s.loc[df_s[attr_max_gain] <= x_v]
        df_high = df_s.loc[df_s[attr_max_gain] > x_v]

        if len(df_low) > self.min_n_data:
            childs['0'] = self.des_tree_nodes(df_low)
        else:
            childs['0'] = df_low['res'].mean()

        if len(df_high) > self.min_n_data:
            childs['1'] = self.des_tree_nodes(df_high)
        else:
            childs['1'] = df_high['res'].mean()

        dct_res['res'] = childs
        return dct_res

    def mse(self, y: pd.Series, y_pred: pd.Series) -> float:
        y_np = y.to_numpy()
        yp_np = y_pred.to_numpy()
        return np.mean((y_np - yp_np)**2)
    
    def get_split_pred(self, df_s: pd.DataFrame, c: str, i: int) -> Tuple[pd.Series, float]:
        """Splits the dataframe at the given index. Returns the predicted value as the mean of the target before and after the split.

        Parameters
        ----------
        df_s : pd.DataFrame
            Input and target data sorted by column c
        c : str
            column that serves as x values
        i : int
            Index for splitting

        Returns
        -------
        Tuple[pd.Series, float]
            pd.Series: Predicted value with the means before and after the split
            float: column value at which the split was performed (average of df_s[c].iloc[i] and df_s[c].iloc[i+1])
        """
        x_split = np.mean([df_s[c].iloc[i-1:i+1]])
        yA_m = df_s['res'].loc[df_s[c] <= x_split].mean()
        yB_m = df_s['res'].loc[df_s[c] > x_split].mean()

        df_s['pred'].loc[df_s[c] <= x_split] = yA_m
        df_s['pred'].loc[df_s[c] > x_split] = yB_m    
        return df_s['pred'], x_split

    def get_min_mse_column(self, df: pd.DataFrame, c: str) -> List:
        """Sorts the dataframe by column c, and returns the index and value of the split necessary to minimize the mean squared error

        Parameters
        ----------
        df : pd.DataFrame
            Input and target data.
        c : str
            Column for which the splits will be performed

        Returns
        -------
        List
            _description_
        """
        mse_res = []
        x_s = []
        df_s = df.sort_values(c)
        for i in range(1,len(df_s)):
            df_s['pred'], x_split = self.get_split_pred(df_s, c, i)
            x_s.append(x_split)
            mse_res.append(self.mse(df_s['res'], df_s['pred']))

        return [x_s[np.argmin(mse_res)],np.min(mse_res)]

    def get_min_mse(self, df: pd.DataFrame) -> Tuple[str,float]:
        """Obtains the column and the split value of the column which allows to minimize the mse

        Parameters
        ----------
        df : pd.DataFrame
            Input and target dat

        Returns
        -------
        Tuple[str,float]
            str: Column that minimizes the mse
            float: Value of df[c] at which the mse is minimized
        """
        cols = [x for x in df.columns if x not in ['res','pred']]
        x_vals = []

        for c in cols:
            x_vals.append(self.get_min_mse_column(df, c))

        xv_np = np.array(x_vals)
        min_index = np.argmin(xv_np[:,1])
        attr = cols[min_index]
        x_v = x_vals[min_index][0]

        return attr, x_v       

In [3]:
class Random_Forest:
    def __init__(self, n_trees = 100) -> None:
        self.random_forest = []
        self.n_trees = n_trees

    def fit(self, X: pd.DataFrame, y: pd.Series, X_cat_cols: List) -> None:
        """Fits a random forest model.  The ensemble of models is saved 
        as the list of trees (dicts) self.random_forest

        Parameters
        ----------
        X : pd.DataFrame
            Input data
        y : pd.Series
            Target data
        X_cat_cols : List
            Categorical columns
        """
        # Bootstrap resample and fitting
        n_samples = len(X)
        self.random_forest = []

        df = X.copy()
        df['res'] = y.copy()

        for i in range(self.n_trees):
            X_bs = df.sample(n_samples,replace=True).reset_index()
            dtc = Decision_Tree_Classifier()
            dtc.fit(X_bs[X.columns], X_bs['res'], X_cat_cols)
            self.random_forest.append(dtc)

    # Single prediction
    def single_predict(self, X: pd.Series) -> int:
        """Returns the predicted value of a single entry of input data

        Parameters
        ----------
        X : pd.Series
            Input data

        Returns
        -------
        int
            Predicted category
        """
        cv = X.to_frame().transpose()
        y_vals = []
        for i, dt in enumerate(self.random_forest):
            y_vals.append(dt.predict(cv).iloc[0])
        return max(set(y_vals), key = y_vals.count)
    
    def predict(self, X: pd.DataFrame) -> pd.Series:
        """Predicts the categories related to the input data X

        Parameters
        ----------
        X : pd.DataFrame
            Input data

        Returns
        -------
        pd.Series
            Predicted categories
        """
        return X.reset_index()[X.columns].apply(lambda x: self.single_predict(x), axis=1)

In [4]:
df = pd.read_csv('../data/1.raw/customerClassification.csv')#, parse_dates=['DateTime'],index_col=['DateTime'])
df.columns
X_cols = ['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size',
       'Segmentation']
X_cat_cols = ['Gender', 'Ever_Married', 'Graduated', 'Profession',
       'Spending_Score', 'Segmentation']
y_col = 'Var_1'

X = df[X_cols]
X = X.fillna(0)

for cat in X_cat_cols:
    X[cat] = X[cat].astype('category').cat.codes
# for c in X.columns:
#     X[c] = utl.min_max_scaling(X[c])[0]

y = df[y_col].fillna(0).astype('category').cat.codes

In [5]:
rf = Random_Forest(n_trees=10)
rf.fit(X, y, X_cat_cols)

In [6]:
y_res = rf.predict(X)

In [7]:
np.set_printoptions(formatter={'float_kind':'{:4.0f}'.format})
cm = utl.confusion_matrix(y, y_res)
acc = utl.accuracy_classification(cm)
pres = utl.presicion_classification(cm, i=4)
rec = utl.recall_classification(cm, i=4)
f1 = utl.fbeta_classification(cm, i=4)

acc, pres, rec, f1

(0.7413237481408032,
 0.40404040404040403,
 0.7612456747404844,
 0.5278944211157768)

In [8]:
cm

array([[  17,    0,    0,    2,    2,    0,   55,    0],
       [   0,   39,    1,    1,    0,    2,   90,    0],
       [   0,    2,   78,    3,   12,    1,  325,    1],
       [   1,    1,    5,  215,   29,    3,  568,    0],
       [   0,    0,    6,   18,  440,    4,  621,    0],
       [   0,    0,    0,    0,    5,   30,   50,    0],
       [   1,    2,    8,   19,   81,    3, 5121,    3],
       [   0,    1,    0,    1,    9,    0,  151,   41]])